
## objectifs : detection de communauté dans un Graphe 
<p> <ol> 
<li> Ecrire l'algorithe de Girvan Newman en python</li> 
<li>visualiser les communaute dans le graphe </li>
<li> appliquer cl'algorithme de Givann Newman disponible dans la library networkx </li>
</ol></p>


### l'algorithme de Girvan en python 